In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np


In [2]:
df= pd.read_excel("./test_data/data.xlsx", sheet_name="Sheet1")
print(df.shape) # "Shape" shows count of columns and rows


(10000, 13)


In [3]:
df.columns

Index(['Unnamed: 0', 'city', 'country', 'customer_id', 'dob',
       'enrollment_date', 'first_name', 'gender', 'last_name', 'lifetime_id',
       'registration_id', 'renewal_date', 'residential address'],
      dtype='object')

This is to replace "NAN" values with a string, helps with some of the formulae later on

In [13]:

df.fillna("", inplace=True)

In [7]:
df_duplicates = df[df.duplicated(['Auth_ID'],keep=False)]["Customer_ID"]
df_duplicates
df_duplicates.to_excel("./output/101_Duplicates.xlsx", sheet_name='Duplicates')
df_deduped_ID= df['Auth_ID'].drop_duplicates()


KeyError: Index(['Auth_ID'], dtype='object')

This is to list the headers

TODO: Check completeness of the file 

In [ ]:
import datetime
#here we sort, not really needed, but makes it easier. The "inplace" parameter makes the changes
#to the df itself, otherwise, it would return a separate data frame
df.sort_values('MEMBER_ID', axis=0, ascending=True, inplace=True)


df["Country"]=""
df["Address"]=""
df['Country'] = df['Residential Address'].map(lambda a: a.split(" ")[-1])
df['Address'] = df['Residential Address'].map(lambda a: " ".join(a.split(" ")[0:-1]))
# this is interesting, when we export to CSV it comes in a different format
#TODO investigate how to get it in full datetime format.


In [ ]:
# This is just a check, listing the results of the previous function
df[["Residential Address","Address","Country"][100:110]

In [ ]:
#df.to_csv("Data_fixed.csv") # if we want to export the csv

In [ ]:
# we retain records that have renewal date after a date, we assign to the df so we are
# effectively overwritting the df with a subset version
# we could opt to have it as df1 or whatever to preserve the original dataset (df)

df= df[df["RENEWAL_DATE"]> pd.to_datetime("2019-10-31")]

Some checks

In [ ]:
df[df["MEMBER_ID"]==""] # looking for empty members

In [ ]:
df[df["RENEWAL_DATE"]==""] # looking for empty renewal dates

In [ ]:
# Extracting a summarisation of registration ID with the count of member ids
df1 = df.groupby('REGISTRATION_ID')['MEMBER_ID'].count().sort_values(ascending=False)
print(df1.shape)
df1.to_excel("./output/Groupby REGISTRATION_ID.xlsx")

In [ ]:
df2 = df.groupby('MEMBER_ID')['MEMBER_ID'].count().sort_values(ascending=False)
list_duplis= df2[df2 >1].keys().to_list()
df[df["MEMBER_ID"].isin (list_duplis)].to_excel("Duplicate Member IDs.xlsx")
df2.to_excel("./output/Groupby MEMBER_ID")
df2[0:10]

In [ ]:
df3= df[df.duplicated(subset=["MEMBER_ID","REGISTRATION_ID"])]
df3.shape[0]

In [ ]:
df4 = df.groupby(['RENEWAL_DATE'])['MEMBER_ID'].count().sort_values(ascending=False)
print(df4[0:10])
print(df4.shape)
df4.to_excel("./output/Groupby RENEWAL_DATE.xlsx")

In [ ]:
df5 = df.groupby(['RENEWAL_DATE','Customer type'])['MEMBER_ID'].count().sort_values(ascending=False)
print(df5[0:10])
print(df5.shape)
df5.to_excel("./output/Groupby RENEWAL_DATE and Customer Type.xlsx")

In [ ]:
df6 = df.groupby(['Country','Customer type','INSURER_NAME'])['MEMBER_ID'].count().sort_values(ascending=False)
print(df6.shape)
df6b= df6.reset_index() #to avoid merged cells in the report
df6b.to_excel("./output/Groupby Country and Customer Type.xlsx")

In [ ]:
df7 = df.groupby(['REGISTRATION_ID','Country',"INSURER_NAME"])['MEMBER_ID'].count().sort_values(ascending=False)
print(df7.shape)
df7b= df7.reset_index()
df7c= df7b[df7b["Country"]!=""]
print(df7c.shape)

df7d= df7c[df7c.duplicated(subset=["REGISTRATION_ID"], keep=False)].sort_values("REGISTRATION_ID")
df7d.shape
df7d.to_excel("./output/Groupby REGISTRATION_ID and country diff address.xlsx", merge_cells=False)

In [ ]:
df8 = df7.reset_index()
pd.pivot_table(df8,index=["Country"],values=["MEMBER_ID"],
               columns=["INSURER_NAME"],aggfunc=[np.sum]).to_excel("./output/Pivot.xlsx")